In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline
from sklearn import naive_bayes
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import spacy
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
import re
from bs4 import BeautifulSoup
import unicodedata
from sklearn.metrics import roc_auc_score, roc_curve, auc, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import SGDClassifier, LogisticRegression
from nltk.corpus import stopwords
import sys
import sklearn
pd.set_option('display.max_rows', None)
df = pd.read_csv('Nega.csv', header= 0,
                        encoding= 'unicode_escape')
print (df.head(10))

  App_name   Category                               User_Review Sentiments  \
0  brainly  Education  Not working properly plz fix the problem   Negative   
1  brainly  Education         App is not good, stuck in between   Negative   
2  brainly  Education                      not working properly   Negative   
3  brainly  Education                        app is not working   Negative   
4  brainly  Education   this app will be not started some issue   Negative   
5  brainly  Education                            so many issues   Negative   
6  brainly  Education                          app is troubling   Negative   
7  brainly  Education                              Its annoying   Negative   
8  brainly  Education                      not working properly   Negative   
9  brainly  Education          it shows brainly not responding.   Negative   

  Review_Type  
0          AC  
1          AC  
2          AC  
3          AC  
4          AC  
5          AC  
6          AC  
7          AC

In [5]:
classes = df['Review_Type']
print(classes.value_counts())

PI    2403
BR    2201
AC    1129
FI     937
CI     873
VU     777
AI     363
FR     223
UI     206
Name: Review_Type, dtype: int64


In [6]:
df.head()

,App_name,Category,User_Review,Sentiments,Review_Type
0,brainly,Education,Not working properly plz fix the problem,Negative,AC
1,brainly,Education,"App is not good, stuck in between",Negative,AC
2,brainly,Education,not working properly,Negative,AC
3,brainly,Education,app is not working,Negative,AC
4,brainly,Education,this app will be not started some issue,Negative,AC


In [7]:
summary = df['Review_Type']
#print(Review[:10])
processed = summary.str.replace(r'^\$?([1-9]{1}[0-9]{0,2}(\,[0-9]{3})*(\.[0-9]{0,2})?|[1-9]{1}[0-9]{0,}(\.[0-9]{0,2})?|0(\.[0-9]{0,2})?|(\.[0-9]{1,2})?)$' , 'moneysymbol')
processed = processed.str.replace(r'(\d)' , '')
processed = processed.str.replace(r'^\D?(\d{3})\D?\D?(\d{3})\D?(\d{4})$' , ' ')
processed = processed.str.replace(r'\s+' , ' ')
processed = processed.str.replace(r'[ \t]+$' , ' ')
processed = processed.str.replace(r'[&#();*]' , ' ')
processed = processed.str.replace(r'[,:]' , ' ')
processed = processed.str.replace(r'[.!?\\-]' , ' ')
processed = processed.str.replace(r'\'' , ' ')
processed = processed.str.lower()
processed = processed.str.replace(r'\s[a-z]\s' , ' ')
processed = processed.str.replace(r'\[[^\]]*\]',' ')
processed = processed.str.replace(r'\"',' ')
processed = processed.str.replace(r'\`',' ')
print(processed)

<ipython-input-7-6c6d7739f473>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  processed = summary.str.replace(r'^\$?([1-9]{1}[0-9]{0,2}(\,[0-9]{3})*(\.[0-9]{0,2})?|[1-9]{1}[0-9]{0,}(\.[0-9]{0,2})?|0(\.[0-9]{0,2})?|(\.[0-9]{1,2})?)$' , 'moneysymbol')
<ipython-input-7-6c6d7739f473>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  processed = processed.str.replace(r'(\d)' , '')
<ipython-input-7-6c6d7739f473>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  processed = processed.str.replace(r'^\D?(\d{3})\D?\D?(\d{3})\D?(\d{4})$' , ' ')
<ipython-input-7-6c6d7739f473>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  processed = processed.str.replace(r'\s+' , ' ')
<ipython-input-7-6c6d7739f473>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  pr

0       ac
1       ac
2       ac
3       ac
4       ac
5       ac
6       ac
7       ac
8       ac
9       ac
10      ac
11      ac
12      ac
13      ac
14      ac
15      ac
16      ac
17      ac
18      ac
19      ac
20      ac
21      ac
22      ac
23      ac
24      ac
25      ac
26      ac
27      ac
28      ac
29      ac
30      ac
31      ac
32      ac
33      ac
34      ac
35      ac
36      ac
37      ac
38      ac
39      ac
40      ac
41      ac
42      ac
43      ac
44      ac
45      ac
46      ac
47      ac
48      ac
49      ac
50      ac
51      ac
52      ac
53      ac
54      ac
55      ac
56      ac
57      ac
58      ac
59      ac
60      ac
61      ac
62      ac
63      ac
64      ac
65      ac
66      ac
67      ac
68      ac
69      ac
70      ac
71      ac
72      ac
73      ac
74      ac
75      ac
76      ac
77      ac
78      ac
79      ac
80      ac
81      ac
82      ac
83      ac
84      ac
85      ac
86      ac
87      ac
88      ac
89      ac
90      ac

<ipython-input-7-6c6d7739f473>:16: FutureWarning: The default value of regex will change from True to False in a future version.
  processed = processed.str.replace(r'\`',' ')


In [8]:
from nltk.corpus import stopwords

In [9]:
stopset = set(stopwords.words('english'))
vectorizer = TfidfVectorizer(use_idf=True, lowercase=True, strip_accents='ascii', stop_words=stopset) 

In [10]:
stop_words = set(stopwords.words('english')) 

In [11]:
processed = processed.apply(lambda x: ' ' .join(term for term in x.split() if term not in stop_words))

In [12]:
print(processed)

0       ac
1       ac
2       ac
3       ac
4       ac
5       ac
6       ac
7       ac
8       ac
9       ac
10      ac
11      ac
12      ac
13      ac
14      ac
15      ac
16      ac
17      ac
18      ac
19      ac
20      ac
21      ac
22      ac
23      ac
24      ac
25      ac
26      ac
27      ac
28      ac
29      ac
30      ac
31      ac
32      ac
33      ac
34      ac
35      ac
36      ac
37      ac
38      ac
39      ac
40      ac
41      ac
42      ac
43      ac
44      ac
45      ac
46      ac
47      ac
48      ac
49      ac
50      ac
51      ac
52      ac
53      ac
54      ac
55      ac
56      ac
57      ac
58      ac
59      ac
60      ac
61      ac
62      ac
63      ac
64      ac
65      ac
66      ac
67      ac
68      ac
69      ac
70      ac
71      ac
72      ac
73      ac
74      ac
75      ac
76      ac
77      ac
78      ac
79      ac
80      ac
81      ac
82      ac
83      ac
84      ac
85      ac
86      ac
87      ac
88      ac
89      ac
90      ac

In [13]:
from nltk.stem import WordNetLemmatizer 

In [14]:
lemmatizer = WordNetLemmatizer() 

In [15]:
processed = processed.apply(lambda x: ' ' .join(lemmatizer.lemmatize(term) for term in x.split()))

In [16]:
processed

0       ac
1       ac
2       ac
3       ac
4       ac
5       ac
6       ac
7       ac
8       ac
9       ac
10      ac
11      ac
12      ac
13      ac
14      ac
15      ac
16      ac
17      ac
18      ac
19      ac
20      ac
21      ac
22      ac
23      ac
24      ac
25      ac
26      ac
27      ac
28      ac
29      ac
30      ac
31      ac
32      ac
33      ac
34      ac
35      ac
36      ac
37      ac
38      ac
39      ac
40      ac
41      ac
42      ac
43      ac
44      ac
45      ac
46      ac
47      ac
48      ac
49      ac
50      ac
51      ac
52      ac
53      ac
54      ac
55      ac
56      ac
57      ac
58      ac
59      ac
60      ac
61      ac
62      ac
63      ac
64      ac
65      ac
66      ac
67      ac
68      ac
69      ac
70      ac
71      ac
72      ac
73      ac
74      ac
75      ac
76      ac
77      ac
78      ac
79      ac
80      ac
81      ac
82      ac
83      ac
84      ac
85      ac
86      ac
87      ac
88      ac
89      ac
90      ac

In [18]:
from nltk.tokenize import word_tokenize

In [19]:
allwords = []
for message in processed:
    words = word_tokenize(message)
    for w in words:
        allwords.append(w)
        
allwords = nltk.FreqDist(allwords)

print('Number of words: {}'.format(len(allwords)))
print('most common words: {}'.format(allwords.most_common(50)))

Number of words: 9
most common words: [('pi', 2403), ('br', 2201), ('ac', 1129), ('fi', 937), ('ci', 873), ('vu', 777), ('ai', 363), ('fr', 223), ('ui', 206)]


In [21]:
word_features = allwords.most_common(50)
word_features = [i[0] for i in word_features]
print(word_features)

['pi', 'br', 'ac', 'fi', 'ci', 'vu', 'ai', 'fr', 'ui']


In [22]:
def find_features(message):
    words = word_tokenize(message)
    features = {}
    for word in word_features:
        features[word] = (word in words)
    return features

features = find_features(processed[0])

for key, value in features.items():
    if value == True:
        print(key)

ac


In [23]:
processed[0]

'ac'

In [25]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:
vectorizer = TfidfVectorizer()

In [27]:
X = vectorizer.fit_transform(processed).toarray()

In [28]:
y = df.Review_Type
print(y)

0       AC
1       AC
2       AC
3       AC
4       AC
5       AC
6       AC
7       AC
8       AC
9       AC
10      AC
11      AC
12      AC
13      AC
14      AC
15      AC
16      AC
17      AC
18      AC
19      AC
20      AC
21      AC
22      AC
23      AC
24      AC
25      AC
26      AC
27      AC
28      AC
29      AC
30      AC
31      AC
32      AC
33      AC
34      AC
35      AC
36      AC
37      AC
38      AC
39      AC
40      AC
41      AC
42      AC
43      AC
44      AC
45      AC
46      AC
47      AC
48      AC
49      AC
50      AC
51      AC
52      AC
53      AC
54      AC
55      AC
56      AC
57      AC
58      AC
59      AC
60      AC
61      AC
62      AC
63      AC
64      AC
65      AC
66      AC
67      AC
68      AC
69      AC
70      AC
71      AC
72      AC
73      AC
74      AC
75      AC
76      AC
77      AC
78      AC
79      AC
80      AC
81      AC
82      AC
83      AC
84      AC
85      AC
86      AC
87      AC
88      AC
89      AC
90      AC

In [29]:
# find categorical variables
categorical = [var for var in df.columns if df[var].dtype=='O']
print('There are {} categorical variables\n'.format(len(categorical)))
print('The categorical variables are :', categorical)
# check for cardinality in categorical variables
for var in categorical:    
    print(var, ' contains ', len(df[var].unique()), ' labels')



There are 5 categorical variables

The categorical variables are : ['App_name', 'Category', 'User_Review', 'Sentiments', 'Review_Type']
App_name  contains  36  labels
Category  contains  1  labels
User_Review  contains  5698  labels
Sentiments  contains  1  labels
Review_Type  contains  9  labels


In [30]:
X = vectorizer.fit_transform(df.User_Review)

In [31]:
print (y.shape)
print (X.shape)


(9112,)
(9112, 7546)


In [38]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20, random_state=0)

In [39]:
clf = naive_bayes.MultinomialNB()
clf.fit(X_train, y_train)
clf.predict(X_test)
y_pred = clf.predict(X_test)

In [40]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


Accuracy: 0.48985189248491495


In [42]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

          AC       0.74      0.28      0.41       227
          AI       1.00      0.01      0.03        72
          BR       0.41      0.85      0.56       443
          CI       0.96      0.14      0.24       166
          FI       0.89      0.08      0.15       201
          FR       0.00      0.00      0.00        43
          PI       0.52      0.82      0.63       484
          UI       0.00      0.00      0.00        40
          VU       0.86      0.12      0.21       147

    accuracy                           0.49      1823
   macro avg       0.60      0.26      0.25      1823
weighted avg       0.62      0.49      0.41      1823



C:\Users\HP\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [43]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
logreg = LogisticRegression(solver='liblinear', random_state=0)
#fit the model
logreg.fit(X_train, y_train)
y_pred_test = logreg.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
#print(classification_report(y_test, y_pred_test)


Accuracy: 0.48985189248491495


In [45]:
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.6829402084476138
              precision    recall  f1-score   support

          AC       0.67      0.75      0.71       227
          AI       0.81      0.86      0.83        72
          BR       0.59      0.70      0.64       443
          CI       0.79      0.61      0.69       166
          FI       0.69      0.33      0.45       201
          FR       0.67      0.47      0.55        43
          PI       0.70      0.77      0.74       484
          UI       0.88      0.72      0.79        40
          VU       0.77      0.76      0.76       147

    accuracy                           0.68      1823
   macro avg       0.73      0.66      0.68      1823
weighted avg       0.69      0.68      0.68      1823



In [108]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25, random_state=0)

In [109]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Create the model with 100 trees
model = RandomForestClassifier(n_estimators=100, 
                               bootstrap = True,
                               max_features = 'sqrt')
# Fit on training data
model.fit(X_train, y_train)
rf_predictions = model.predict(X_test)
#testing set accuracy
print("Accuracy:",metrics.accuracy_score(y_test, rf_predictions))
print(classification_report(y_test, rf_predictions))

Accuracy: 0.7431957857769974
              precision    recall  f1-score   support

          AC       0.84      0.85      0.84       277
          AI       0.87      0.80      0.83        98
          BR       0.64      0.76      0.69       550
          CI       0.89      0.75      0.81       213
          FI       0.80      0.38      0.51       248
          FR       0.75      0.78      0.77        59
          PI       0.72      0.79      0.75       603
          UI       0.90      0.78      0.84        49
          VU       0.79      0.81      0.80       181

    accuracy                           0.74      2278
   macro avg       0.80      0.74      0.76      2278
weighted avg       0.75      0.74      0.74      2278



In [110]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=0)

In [111]:
clf = naive_bayes.MultinomialNB()
clf.fit(X_train, y_train)
clf.predict(X_test)
y_pred = clf.predict(X_test)

In [112]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.48985189248491495


In [113]:
predictions = clf.predict(X_test)

In [114]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,predictions))  
print(classification_report(y_test,predictions))  
print(accuracy_score(y_test, predictions))

[[ 64   0 114   0   0   0  49   0   0]
 [  0   1   8   0   0   0  63   0   0]
 [  4   0 375   0   1   0  63   0   0]
 [  8   0  98  23   0   0  37   0   0]
 [  4   0 101   0  16   0  79   0   1]
 [  2   0  13   0   0   0  28   0   0]
 [  4   0  83   0   1   0 396   0   0]
 [  0   0  28   0   0   0  10   0   2]
 [  1   0  84   1   0   0  43   0  18]]
              precision    recall  f1-score   support

          AC       0.74      0.28      0.41       227
          AI       1.00      0.01      0.03        72
          BR       0.41      0.85      0.56       443
          CI       0.96      0.14      0.24       166
          FI       0.89      0.08      0.15       201
          FR       0.00      0.00      0.00        43
          PI       0.52      0.82      0.63       484
          UI       0.00      0.00      0.00        40
          VU       0.86      0.12      0.21       147

    accuracy                           0.49      1823
   macro avg       0.60      0.26      0.25      182

C:\Users\HP\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [115]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=0)

In [116]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
logreg = LogisticRegression(solver='liblinear', random_state=0)
#fit the model
logreg.fit(X_train, y_train)
y_pred_test = logreg.predict(X_test)

In [117]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_pred_test))  
print(classification_report(y_test,y_pred_test))  
print(accuracy_score(y_test, y_pred_test))

[[144   0  50   4   3   1  19   0   6]
 [  0  50   2   0   2   0  18   0   0]
 [ 19   0 327  13   4   0  70   1   9]
 [  9   0  40  84   5   0  26   0   2]
 [  8   0  65   2  59   7  51   0   9]
 [  3   0  10   3   0  13  13   0   1]
 [ 13   8  60   6   4   0 389   0   4]
 [  0   0  15   1   0   0   4  17   3]
 [  2   3  17   0   3   0  16   0 106]]
              precision    recall  f1-score   support

          AC       0.73      0.63      0.68       227
          AI       0.82      0.69      0.75        72
          BR       0.56      0.74      0.64       443
          CI       0.74      0.51      0.60       166
          FI       0.74      0.29      0.42       201
          FR       0.62      0.30      0.41        43
          PI       0.64      0.80      0.71       484
          UI       0.94      0.42      0.59        40
          VU       0.76      0.72      0.74       147

    accuracy                           0.65      1823
   macro avg       0.73      0.57      0.61      182

In [118]:
from sklearn import svm
#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


Accuracy: 0.6829402084476138


In [119]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=0)

In [120]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Create the model with 100 trees
model = RandomForestClassifier(n_estimators=100, 
                               bootstrap = True,
                               max_features = 'sqrt')
# Fit on training data
model.fit(X_train, y_train)
rf_predictions = model.predict(X_test)
#testing set accuracy
print("Accuracy:",metrics.accuracy_score(y_test, rf_predictions))
print(classification_report(y_test, rf_predictions))

Accuracy: 0.731528895391368
              precision    recall  f1-score   support

          AC       0.82      0.84      0.83       333
          AI       0.87      0.78      0.83       106
          BR       0.61      0.76      0.68       654
          CI       0.92      0.72      0.81       271
          FI       0.78      0.36      0.50       286
          FR       0.79      0.71      0.75        73
          PI       0.70      0.77      0.74       731
          UI       0.92      0.76      0.83        59
          VU       0.80      0.81      0.80       221

    accuracy                           0.73      2734
   macro avg       0.80      0.73      0.75      2734
weighted avg       0.75      0.73      0.73      2734



In [121]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25, random_state=0)

In [122]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Create the model with 100 trees
model = RandomForestClassifier(n_estimators=100, 
                               bootstrap = True,
                               max_features = 'sqrt')
# Fit on training data
model.fit(X_train, y_train)
rf_predictions = model.predict(X_test)
#testing set accuracy
print("Accuracy:",metrics.accuracy_score(y_test, rf_predictions))
print(classification_report(y_test, rf_predictions))

Accuracy: 0.7418788410886743
              precision    recall  f1-score   support

          AC       0.85      0.85      0.85       277
          AI       0.88      0.79      0.83        98
          BR       0.62      0.77      0.69       550
          CI       0.91      0.74      0.81       213
          FI       0.81      0.38      0.51       248
          FR       0.77      0.78      0.77        59
          PI       0.72      0.79      0.75       603
          UI       0.90      0.76      0.82        49
          VU       0.80      0.80      0.80       181

    accuracy                           0.74      2278
   macro avg       0.81      0.74      0.76      2278
weighted avg       0.76      0.74      0.74      2278



In [123]:
from sklearn import svm
#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


Accuracy: 0.6725197541703248


In [124]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
logreg = LogisticRegression(solver='liblinear', random_state=0)
#fit the model
logreg.fit(X_train, y_train)
y_pred_test = logreg.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6725197541703248
